# Feature Engineering Techniques

Shoutout to Greg Hogg for his great video on youtube

https://www.youtube.com/watch?v=GduT2ZCc26E&t=40s


## Downloading the dataset

In [5]:
# download the dataset either using your kaggle API
#!kaggle datasets download -d camnugent/california-housing-prices
#!unzip california-housing-prices.zip

# OR manually from this link
# https://www.kaggle.com/datasets/camnugent/california-housing-prices


In [450]:
import pandas as pd
import numpy as np


In [446]:
df = pd.read_csv('housing.csv')

# immediately drop null values
df.dropna(inplace=True)

df


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [447]:
# train test split the dataframe
df = df.sample(frac=1, random_state=23)
train_size = int(len(df) * 0.8)

train_df = df[:train_size]

test_df = df[train_size:]

train_df.reset_index(drop=True, inplace=True)
train_df


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-119.89,36.73,43.0,524.0,93.0,302.0,93.0,2.6146,81300.0,INLAND
1,-117.25,34.13,37.0,2498.0,472.0,1291.0,487.0,3.0000,83400.0,INLAND
2,-117.10,32.68,49.0,1412.0,350.0,1200.0,332.0,2.0398,93600.0,NEAR OCEAN
3,-121.96,37.34,36.0,844.0,153.0,373.0,160.0,5.7910,254100.0,<1H OCEAN
4,-121.24,36.34,33.0,1691.0,308.0,792.0,262.0,2.6648,164600.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
16341,-121.42,38.54,29.0,2358.0,493.0,1071.0,470.0,2.9250,94300.0,INLAND
16342,-117.73,34.05,36.0,975.0,243.0,809.0,233.0,2.8929,118100.0,INLAND
16343,-116.31,34.13,20.0,2352.0,556.0,1217.0,481.0,1.6063,55400.0,INLAND
16344,-119.80,36.75,41.0,1659.0,466.0,1391.0,447.0,1.3527,61200.0,INLAND


In [448]:
y_train = train_df['median_house_value'].to_numpy()
y_test = test_df['median_house_value'].to_numpy()

y_train.shape, y_test.shape


((16346,), (4087,))

### Baseline MAE

In [454]:
from sklearn.metrics import mean_absolute_error

# a baseline prediction using the average price for all samples
avg_median_house_val = y_train.mean()
baseline_predictions = np.full(y_test.shape, fill_value=avg_median_house_val)

mean_absolute_error(baseline_predictions, y_test)


91817.08456221559

### PCA (Linear Dimension Reduction)

In [456]:
train_df[['total_rooms', 'total_bedrooms', 'households']].corr()


,total_rooms,total_bedrooms,households
total_rooms,1.000000,0.930416,0.919786
total_bedrooms,0.930416,1.000000,0.980438
households,0.919786,0.980438,1.000000


In [458]:
# test a model with these 3 features before applying PCA

X_train_rooms = train_df[['total_rooms', 'total_bedrooms', 'households']].to_numpy()


X_test_rooms = test_df[['total_rooms', 'total_bedrooms', 'households']].to_numpy()

X_train_rooms.shape, X_test_rooms.shape


((16346, 3), (4087, 3))

In [459]:
from sklearn.ensemble import RandomForestRegressor

forest_basemodel = RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1)
forest_basemodel.fit(X_train_rooms, y_train)


forest_basemodel_preds = forest_basemodel.predict(X_test_rooms)

# before PCA
mean_absolute_error(forest_basemodel_preds, y_test)


81379.19673403125

In [460]:
# now let's test again after pca
from sklearn.decomposition import PCA

# n_components is the number of output features
# we will basically feed in 3 features and get 2 instead
pca = PCA(n_components=2)

pca.fit(X_train_rooms)

X_train_pca = pca.transform(X_train_rooms)
X_test_pca = pca.transform(X_test_rooms)

X_train_pca.shape, X_test_pca.shape


((16346, 2), (4087, 2))